In [0]:
from pyspark.sql.functions import first, rand, count
dates = spark.sql("SELECT explode(sequence(DATE'2024-01-01', DATE'2024-03-24', INTERVAL 1 DAY)) as  calendar_date")
c_id = spark.sql("SELECT explode(sequence(1,200, 1)) as  client_id")
types = spark.sql("""SELECT concat("col_", colName) as col_name from (SELECT explode(sequence(1,20, 1)) as  colName)""")
 
dates = dates.repartition(99)
c_id = c_id.repartition(11)
types = types.repartition(1)
 
df_cartesian = c_id.crossJoin(dates.select("calendar_date")).crossJoin(types.select("col_name")).select("client_id","calendar_date","col_name")
df_cartesian2 = df_cartesian.groupBy("calendar_date").agg(count("client_id"))
 
# display(df_cartesian2.limit(1000))
 
df_cartesian = df_cartesian.withColumn("val", (rand()*10).cast("int"))

df_grp = df_cartesian.groupBy("client_id","calendar_date").pivot("col_name").agg((first("val").alias("val")))

display(df_grp)


In [0]:
from pyspark.sql.functions import rand, first, count

dates = spark.sql("SELECT explode(sequence(DATE'2024-01-01', DATE'2024-03-24', INTERVAL 1 DAY)) as  calendar_date")
c_id = spark.sql("SELECT explode(sequence(1,200, 1)) as  client_id")
types = spark.sql("""SELECT concat("col_", colName) as col_name from (SELECT explode(sequence(1,20, 1)) as  colName)""")

dates = dates.repartition(99)
c_id = c_id.repartition(11)
types = types.repartition(1)

df_cartesian = c_id.crossJoin(dates.select("calendar_date")).crossJoin(types.select("col_name"))
df_cartesian = df_cartesian.withColumn("val", (rand()*10).cast("int"))

df_grp1 = df_cartesian.groupBy("client_id","calendar_date").pivot("col_name").agg(first("val"))
df_grp1 = df_grp1.withColumn("table_id", (rand()*1000000).cast("int"))

df_table1 = df_grp1.limit(1000000)

df_grp2 = df_cartesian.groupBy("client_id","calendar_date").pivot("col_name").agg(first("val"))
df_grp2 = df_grp2.withColumn("table_id", (rand()*1000000).cast("int"))

df_table2 = df_grp2.limit(1000000)

# INNER JOIN
df_inner = df_table1.join(df_table2, ["client_id", "calendar_date"], "inner")

# LEFT JOIN
df_left = df_table1.join(df_table2, ["client_id", "calendar_date"], "left")

# usuwanie duplikatów - dwie funkcje
df_inner_dedup = df_inner.distinct()
df_left_dedup = df_left.dropDuplicates()

df_inner_dedup.show(10)
df_left_dedup.show(10)

+---------+-------------+-----+------+------+------+------+------+------+------+------+------+------+-----+------+-----+-----+-----+-----+-----+-----+-----+--------+-----+------+------+------+------+------+------+------+------+------+------+-----+------+-----+-----+-----+-----+-----+-----+-----+--------+
|client_id|calendar_date|col_1|col_10|col_11|col_12|col_13|col_14|col_15|col_16|col_17|col_18|col_19|col_2|col_20|col_3|col_4|col_5|col_6|col_7|col_8|col_9|table_id|col_1|col_10|col_11|col_12|col_13|col_14|col_15|col_16|col_17|col_18|col_19|col_2|col_20|col_3|col_4|col_5|col_6|col_7|col_8|col_9|table_id|
+---------+-------------+-----+------+------+------+------+------+------+------+------+------+------+-----+------+-----+-----+-----+-----+-----+-----+-----+--------+-----+------+------+------+------+------+------+------+------+------+------+-----+------+-----+-----+-----+-----+-----+-----+-----+--------+
|        1|   2024-01-01|    3|     3|     6|     5|     2|     3|     3|     0|  